In [38]:
%run conf_files.ipynb

In [32]:
import pandas as pd                  # for DataFrames
from opencage.geocoder import OpenCageGeocode
key = 'd1dca54adaf64d419fa1a6a6efef0e43'
geocoder = OpenCageGeocode(key)

In [33]:
#build functions
from sklearn.neighbors import BallTree
import numpy as np

def get_nearest(src_points, candidates, k_neighbors=10):
    """Find nearest neighbors for all source points from a set of candidate points"""

    # Create tree from the candidate points
    tree = BallTree(candidates, leaf_size=15, metric='haversine')

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()
    radius = distances[9]

    # Get closest indices and distances (i.e. array at index 0)
    # note: for the second closest points, you would take index 1, etc.
    closest0 = indices[0]    
    closest1 = indices[1]
    closest2 = indices[2]
    closest3 = indices[3]
    closest4 = indices[4]
    closest5 = indices[5]
    closest6 = indices[6]
    closest7 = indices[7]
    closest8 = indices[8]
    closest9 = indices[9]
    #return (closest9)
    # Return indices and distances
    return (radius, closest0, closest1,closest2,closest3,closest4,closest5,closest6,closest7,closest8,closest9)

In [34]:
def nearest_neighbor(left_gdf, right_gdf, return_dist=False):
    """
    For each point in left_gdf, find closest point in right GeoDataFrame and return them.

    NOTICE: Assumes that the input Points are in WGS84 projection (lat/lon).
    """

    left_geom_col = left_gdf.geometry.name
    right_geom_col = right_gdf.geometry.name

    # Ensure that index in right gdf is formed of sequential numbers
    right = right_gdf.copy().reset_index(drop=True)

    # Parse coordinates from points and insert them into a numpy array as RADIANS
    left_radians = np.array(left_gdf[left_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())
    right_radians = np.array(right[right_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())

    # Find the nearest points
    # -----------------------
    # closest ==> index in right_gdf that corresponds to the closest point
    # dist ==> distance between the nearest neighbors (in meters)

    radius, closest, closest1,closest2,closest3,closest4,closest5,closest6,closest7,closest8,closest9 = get_nearest(src_points=left_radians, candidates=right_radians)
    #closest = get_nearest(src_points=left_radians, candidates=right_radians)
    # Return points from right GeoDataFrame that are closest to points in left GeoDataFrame
    closest_points = right.loc[closest].reset_index(drop=True)
    closest_points1 = right.loc[closest1].reset_index(drop=True)
    closest_points2 = right.loc[closest2].reset_index(drop=True)
    closest_points3 = right.loc[closest3].reset_index(drop=True)
    closest_points4 = right.loc[closest4].reset_index(drop=True)
    closest_points5 = right.loc[closest5].reset_index(drop=True)
    closest_points6 = right.loc[closest6].reset_index(drop=True)
    closest_points7 = right.loc[closest7].reset_index(drop=True)
    closest_points8 = right.loc[closest8].reset_index(drop=True)
    closest_points9 = right.loc[closest9].reset_index(drop=True)
    
    df_to_agg = pd.concat([closest_points, closest_points1,
                          closest_points2,
                          closest_points3,
                          closest_points4,
                          closest_points5,
                          closest_points6,
                          closest_points7,
                          closest_points8,
                          closest_points9])
    df_to_agg = df_to_agg.drop(columns = ['geometry'])
    df_agg = df_to_agg.groupby(df_to_agg.index).agg('mean')
    
    # Add distance if requested
    if return_dist:
        # Convert to meters from radians
        earth_radius = 6371000  # meters
        df_agg['radius'] = radius * earth_radius

    merged_gpd = left_gdf.join(df_agg)
    return merged_gpd

In [39]:
#covid_dt = pd.read_csv("%s/merged_covid_cov_dt_LA.csv" %path_output)
covid_dt = pd.read_csv("%s/merged_covid_dt.csv" %path_out)
covid_dt

,Unnamed: 0,Code,Mean_ann_earnings,median_age_2018,Name,2018 people per sq. km,deaths
0,1,E06000001,25985.0,41.8,Hartlepool,997,20
1,2,E06000002,22878.0,36.2,Middlesbrough,2608,60
2,3,E06000003,23236.0,45.0,Redcar and Cleveland,558,26
3,4,E06000004,26622.0,40.4,Stockton-on-Tees,962,26
4,0,E06000005,26908.0,43.1,Darlington,540,14
...,...,...,...,...,...,...,...
329,331,W06000020,25755.0,42.4,Torfaen,740,33
330,332,W06000021,32558.0,48.6,Monmouthshire,111,25
331,333,W06000022,24974.0,38.8,Newport,805,61
332,318,W06000023,22421.0,49.9,Powys,26,18


In [40]:
#loop to get lat long
list_lat = []   # create empty lists

list_lon = []

for index, row in covid_dt.iterrows(): # iterate over rows in dataframe
    
    query = row['Name']  + ', England, UK'

    results = geocoder.geocode(query)   
    lat = results[0]['geometry']['lat']
    lon = results[0]['geometry']['lng']

    list_lat.append(lat)
    list_lon.append(lon)

# create new columns from lists    

covid_dt['lat'] = list_lat   

covid_dt['lon'] = list_lon

In [41]:
covid_dt.head()

,Unnamed: 0,Code,Mean_ann_earnings,median_age_2018,Name,2018 people per sq. km,deaths,lat,lon
0,1,E06000001,25985.0,41.8,Hartlepool,997,20,54.685728,-1.209370
1,2,E06000002,22878.0,36.2,Middlesbrough,2608,60,54.576042,-1.234405
2,3,E06000003,23236.0,45.0,Redcar and Cleveland,558,26,54.567906,-1.005496
3,4,E06000004,26622.0,40.4,Stockton-on-Tees,962,26,54.564094,-1.312916
4,0,E06000005,26908.0,43.1,Darlington,540,14,54.524208,-1.555581


In [42]:
covid_dt.to_csv("data_out/merged_covid_cov_dt_LA_LL.csv", index=False)

 Now, merge the covid dataset to the modelled background pollution dataset

In [43]:
pm25_df = pd.read_csv("%s/processed_pm25_lonlat.csv" %path, usecols=['pm25_lon','pm25_lat','pm25_val'])
no2_df = pd.read_csv('%s/processed_no2_lonlat.csv' %path, usecols = ['no2_lon','no2_lat','no2_val'])
o3_df = pd.read_csv('%s/processed_o3_lonlat.csv' %path, usecols = ['o3_lon','o3_lat','o3_val'])
pm10_df = pd.read_csv('%s/processed_pm10_lonlat.csv' %path, usecols = ['pm10_lon','pm10_lat','pm10_val'])
so2_df = pd.read_csv('%s/processed_so2_lonlat.csv' %path, usecols = ['so2_lon','so2_lat','so2_val'])
nox_df = pd.read_csv('%s/processed_nox_lonlat.csv' %path, usecols = ['nox_lon','nox_lat','nox_val'])

In [ ]:
#covid_dt = pd.read_csv('%/merged_covid_cov_dt_LA_LL.csv' %path_out)

In [45]:
def to_gpd_air_dt(input_df):
    '''
    from pd to gpd 
    the input datasets must be lon, lat, val
    '''
    out_gpd = gpd.GeoDataFrame(
    input_df, geometry=gpd.points_from_xy(input_df.iloc[:,0], input_df.iloc[:,1]))
    return(out_gpd)

^C


ModuleNotFoundError: No module named 'geopandas'

In [ ]:
pm25_gpd = to_gpd_air_dt(pm25_df)
no2_gpd = to_gpd_air_dt(no2_df)
o3_gpd = to_gpd_air_dt(o3_df)
pm10_gpd = to_gpd_air_dt(pm10_df)
so2_gpd = to_gpd_air_dt(so2_df)
nox_gpd = to_gpd_air_dt(nox_df)

In [ ]:
pm25_covid = nearest_neighbor(covid_gpd[['Code','geometry']], pm25_gpd, return_dist=True)
no2_covid = nearest_neighbor(covid_gpd[['Code','geometry']], no2_gpd, return_dist=True)
o3_covid = nearest_neighbor(covid_gpd[['Code','geometry']], o3_gpd, return_dist=True)
pm10_covid = nearest_neighbor(covid_gpd[['Code','geometry']], pm10_gpd, return_dist=True)
so2_covid = nearest_neighbor(covid_gpd[['Code','geometry']], so2_gpd, return_dist=True)
nox_covid = nearest_neighbor(covid_gpd[['Code','geometry']], nox_gpd, return_dist=True)

In [ ]:

# now i need to match the df above
pm25_covid.head()

In [ ]:
# This code is now obsolete
# pm25_covid = covid_gpd[['Code','geometry']].join(pm25_covid)
# no2_covid = covid_gpd[['Code','geometry']].join(no2_covid)
# o3_covid = covid_gpd[['Code','geometry']].join(o3_covid)
# pm10_covid = covid_gpd[['Code','geometry']].join(pm10_covid)
# so2_covid = covid_gpd[['Code','geometry']].join(so2_covid)
# nox_covid = covid_gpd[['Code','geometry']].join(nox_covid)
# so2_covid

In [ ]:
no2_covid.head()

In [ ]:
#merge everything togerther and export 

covid_Allair_poll_df = pd.merge(covid_gpd.drop(columns=['geometry','Name']), pm25_covid.drop(columns=['geometry','radius']), on='Code')
covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, no2_covid.drop(columns=['geometry','radius']), on='Code')
covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, o3_covid.drop(columns=['geometry','radius']), on='Code')
covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, pm10_covid.drop(columns=['geometry','radius']), on='Code')
covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, so2_covid.drop(columns=['geometry','radius']), on='Code')
covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, nox_covid.drop(columns=['geometry','radius']), on='Code')

covid_Allair_poll_df.to_csv("data_output_v4/merged_covidAir_cov_dt_LA.csv", index=False)

In [ ]:
# I will overwrite on the same variables for simplicity
covid_dt = pd.read_csv('data_output_v4/merged_covid_cov_dt_LA_LL.csv')
# AP5Y_dt = pd.read_csv('data_output_v4/5Yaverage_AP_PCMdata_na.csv')
pm25_df = pd.read_csv("data_output_v4/processed_allAP_lonlat.csv", usecols=['lon','lat','pm25_5yAvg'])
no2_df = pd.read_csv('data_output_v4/processed_allAP_lonlat.csv', usecols = ['lon','lat','no2_5yAvg'])
o3_df = pd.read_csv('data_output_v4/processed_allAP_lonlat.csv', usecols = ['lon','lat','o3_5yAvg'])
pm10_df = pd.read_csv('data_output_v4/processed_allAP_lonlat.csv', usecols = ['lon','lat','pm10_5yAvg'])
# so2_df = pd.read_csv('data_output_v4/processed_allAP_lonlat.csv', usecols = ['lon','lat','so2_val'])
nox_df = pd.read_csv('data_output_v4/processed_allAP_lonlat.csv', usecols = ['lon','lat','nox_5yAvg'])

In [ ]:
import geopandas as gpd
def to_gpd_air_dt(input_df):
    '''
    from pd to gpd 
    the input datasets must be lon, lat, val
    '''
    out_gpd = gpd.GeoDataFrame(
    input_df, geometry=gpd.points_from_xy(input_df.loc[:,'lon'], input_df.loc[:,'lat']))
    return(out_gpd)


In [ ]:
# annotate data type
pm25_gpd = to_gpd_air_dt(pm25_df)
no2_gpd = to_gpd_air_dt(no2_df)
o3_gpd = to_gpd_air_dt(o3_df)
pm10_gpd = to_gpd_air_dt(pm10_df)
# so2_gpd = to_gpd_air_dt(so2_df)
nox_gpd = to_gpd_air_dt(nox_df)

In [ ]:
covid_dt = pd.read_csv('data_v4/merged_covid_cov_dt_LA_LL.csv')
covid_gpd = to_gpd_air_dt(covid_dt)

In [ ]:
#Calculations here 
pm25_covid = nearest_neighbor(covid_gpd[['Code','geometry']], pm25_gpd, return_dist=True)
no2_covid = nearest_neighbor(covid_gpd[['Code','geometry']], no2_gpd, return_dist=True)
o3_covid = nearest_neighbor(covid_gpd[['Code','geometry']], o3_gpd, return_dist=True)
pm10_covid = nearest_neighbor(covid_gpd[['Code','geometry']], pm10_gpd, return_dist=True)
# so2_covid = nearest_neighbor(covid_gpd[['Code','geometry']], so2_gpd, return_dist=True)
nox_covid = nearest_neighbor(covid_gpd[['Code','geometry']], nox_gpd, return_dist=True)

In [ ]:
#merge everything togerther and export 

covid_Allair_poll_df = pd.merge(covid_gpd.drop(columns=['geometry','Name','lon','lat']), 
                                pm25_covid.drop(columns=['geometry','radius','lon','lat','radius']), on='Code')
covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, 
                                no2_covid.drop(columns=['geometry','radius','lon','lat','radius']), on='Code')
covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, 
                                o3_covid.drop(columns=['geometry','radius','lon','lat','radius']), on='Code')
covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, 
                                pm10_covid.drop(columns=['geometry','radius','lon','lat','radius']), on='Code')
# covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, so2_covid.drop(columns=['geometry','radius']), on='Code')
covid_Allair_poll_df = pd.merge(covid_Allair_poll_df, 
                                nox_covid.drop(columns=['geometry','radius','lon','lat','radius']), on='Code')

covid_Allair_poll_df = covid_Allair_poll_df.drop(columns=['Unnamed: 0'])
covid_Allair_poll_df.to_csv("data_output_v4/merged_covidAir_cov_LA_5YA.csv", index=False)

In [ ]:
covid_Allair_poll_df.head()